In [ ]:
# This reloads python code imported from *.py files automatically
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tqdm 
import tqdm.notebook

import optimization as opt
import smooth as optclean

import time
# optimal solution
from optimal_solution import *
import clustering as clus
from util import *

In [ ]:
from scipy.cluster.vq import kmeans,vq,whiten
import itertools

## Initialization

In [ ]:
#n_iter = 5000
n_iter = 10000
# parameters
l = 0.8 # lambda in [0,1], or in [0,2] for G2
beta = 20
# initialization
N = 1000
n_selected = 0
g = G1(N)
# g = G2(N)
f_test=0
brute_force_f=0

## Comparison

In [ ]:
## betas=[5,10,20,50,100]
betas=[40]
nb_strategies=6
f_st=np.zeros((nb_strategies+4,n_iter))
all_results=np.zeros((nb_strategies+4,N))
fig=plt.plot


selected_cities_init=np.zeros(N)

#### Strategy 0 --> 3
for i in range(nb_strategies):
    np.random.seed()
    print('strategy n° '+str(i))
    selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = opt.optimize(g, l, beta=betas[0],  
                                              n_iter=n_iter,mutation_strategy=i,
                                              verbose=False)
    if type(selected_cities_n) == list:
        selected_cities_n = selected_cities_n[-1]
    loss_values[-1]=(loss_value_convex) 
    f_st[i]=loss_values
    plt.plot(f_st[i],label = "mutation_strategy="+str(i))
    all_results[i]=selected_cities_n_convex
    
# for i in range(nb_strategies-2,nb_strategies):
#     np.random.seed()
#     print('strategy n° '+str(i))
#     selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize(g, l, beta=betas[0], n_iter=n_iter, mutation_strategy=i, initial_selection_probability=0.5, precompute_pairwise_dist=False, verbose=True, selected_cities=selected_cities_init)
#     if type(selected_cities_n) == list:
#         selected_cities_n = selected_cities_n[-1]
#     loss_values[-1]=(loss_value_convex) 
#     f_st[i]=loss_values
#     plt.plot(f_st[i],label = "mutation_strategy="+str(i))
#     all_results[i]=selected_cities_n_convex
    
#combining 3 and 2  
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex =opt.optimize_combine(g, l, selected_cities_init, betas=betas, n_iter=n_iter, precompute_pairwise_dist=False, verbose=True)
f_st[nb_strategies]=loss_values
all_results[nb_strategies]=selected_cities_n_convex
plt.plot(f_st[nb_strategies],label = "Combining")
############

#clustering1
selected_cities_n_convex, loss_values,_ =clus.optimize(g, l, beta=20, beta_last=50, n_iter=n_iter, mutation_strategy=1, verbose=False, show=False)
f_st[nb_strategies+1]=loss_values
all_results[nb_strategies+1]=selected_cities_n_convex
plt.plot(f_st[nb_strategies+1],label = "Clustering 1")
#clustering2
print('clustering')
selected_cities_n_convex, loss_values,_ =clus.optimize_simple(g, l, beta=20, n_iter=n_iter, mutation_strategy=1, verbose=False, show=False)
f_st[nb_strategies+2]=loss_values
all_results[nb_strategies+2]=selected_cities_n_convex
plt.plot(f_st[nb_strategies+2],label = "Clustering 2")


#optimization_clean
print('5 with optimize clean')
selected_cities_n,loss_values,_=optclean.optimize(g, l, beta=betas[0], n_iter=n_iter, verbose=True)
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
# f = objective_function_simple(N, l, g, selected_cities_n,pairwise_distances=None)
# print('f found='+str(f))
# f,_,_,_=objective_function_(N, l, g, None, selected_cities_n, None, None)
# print('f found='+str(f))
# print('f found='+str(loss_values[-1]))
f_st[nb_strategies+3]=loss_values

all_results[nb_strategies+3]=selected_cities_n
plt.plot(f_st[nb_strategies+2],label = "Smooth 5")



# plt.hlines(f_test, 0, n_iter, linestyles="--", label="brut-force solution")

plt.legend()
print(f_st[:,-1])

# print('Min for all the methods:')
# #print(np.amin(f_st,1))
# print(np.amin(f_st[:,-1]))
# print(np.argmin(f_st[:,-1]))
#print(np.amin(f_st))
best_min_over_all=np.amin(f_st)
best_min=np.amin(f_st[:,-1])
if best_min_over_all!=best_min:
    print('best was not the last value: best_min= '+str(best_min)+" best_min_over_all= "+str(best_min_over_all))
else:
    print('best was the last value')
# best=np.argmin(np.amin(f_st,1)
best=np.argmin((f_st[:,-1]))
print("The best cost is=%f, found for strategy %i"%(best_min,best))
# print(f'Approximated Bruteforce best solution {best_f}')

plotResult(g,0,all_results[best],None,f_st[best],None,None,verbose=False)

    
np.save('solbest', all_results[best])



In [ ]:
# just to be sure
f = objective_function_simple(N, l, g, all_results[best],
                          pairwise_distances=None)
print(f)

# plotResult(g,0,selected_cities_n_result,selected_cities_n_convex,loss_values,loss_value_convex,None)

In [ ]:
selected_cities_n,loss_values,_=optclean.optimize(g, l, beta=40, n_iter=n_iter)
f = objective_function_simple(g.x.shape[0], l, g, np.array(selected_cities_n[-1]),pairwise_distances=None)
print('f found='+str(f))
f,_,_,_=objective_function_(g.x.shape[0], l, g, None, selected_cities_n, None, None)
print('f found='+str(f))
print('f found='+str(loss_values[-1]))

In [ ]:
selected_cities_init=np.zeros(N)
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize(g, l, beta=betas[0],  
                                          n_iter=n_iter,mutation_strategy=3, initial_selection_probability=0,
                                          precompute_pairwise_dist=False, verbose=False,selected_cities=selected_cities_init)
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
loss_values[-1]=(loss_value_convex) 
plotResult(g,0,selected_cities_n,selected_cities_n_convex,loss_values,loss_value_convex,None,verbose=False)
plt.savefig('conve_hull.png')
